## Module Specifications - quantumcode.py

### This module implements the CHSH Inquality via IBM's quantum computing infrastructure.

It is intended to be executed as an application. Functionality includes preparing
a 2-qubit quantum circuit via qiskit, implementing the CHSH inequality, and running
it on the IBM quantum infrastructure via API.

All code sourced from https://learning.quantum.ibm.com/tutorial/chsh-inequality

#### Importing all necessary libraries to fix the reference errors in quantumcode.py

In [12]:
# importing numpy
import numpy as np

# Qiskit imports
from qiskit import QuantumCircuit
from qiskit.circuit import Parameter
from qiskit.quantum_info import SparsePauliOp

# Qiskit Runtime imports
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import EstimatorV2 as Estimator

# Plotting routines
import matplotlib.pyplot as plt
import matplotlib.ticker as tck

#### Few of the improvements that can be added to the code

*1.*: Add exception handling.

*2.*: Consider parameterization for customized execution.

*3.*: Add output handling. Data should be saved as files or handed off to another
        module for posterity and future analysis.

*4.*: Add a naming scheme for files being saved so that new ones will not overwrite
        old files with the same name.

*5.*: Consider placing the main routine in a function for greater control over execution

In [ ]:
#Addressing the 'FIX: QuantumCircuit referenced but never defined' - fixed by importing necessary libraries

# Define the parameter theta
theta = Parameter("$\\theta$")

chsh_circuit = QuantumCircuit(2)    # Initialize a quantum circut with 2 qubits
chsh_circuit.h(0)                   # Add a hadamard gate to the first qubit
chsh_circuit.cx(0, 1)               # Add controlled-NOT gate to both qubits
chsh_circuit.ry(theta, 0)           # Adds RY gate with theta representing rotation of the qubit's state around the Y-axis

# Render a visual representation of the active circuit.
# Documentation (including parameters details) can be found
# at: https://docs.quantum.ibm.com/guides/visualize-circuits
chsh_circuit.draw(output="mpl", idle_wires=False, style="iqp")

In [15]:
# Designate the total number of phases we are interested in
number_of_phases = 21

# Arrange the phases into a numpy array
phases = np.linspace(0, 2 * np.pi, number_of_phases)

#Addressing the 'FIX: This object is created, but never referenced' - fixed by importing necessary libraries
# "Phases need to be expressed as list of lists in order to work":
# https://learning.quantum.ibm.com/tutorial/chsh-inequality#create-a-list-of-phase-values-to-be-assigned-later
# There are other ways to handle this issue if clarity becomes a problem
individual_phases = [[ph] for ph in phases]

In [16]:
#Addressing the 'FIX:   SparsePauliOp is referenced, but never defined' - fixed by importing necessary libraries

# Create a quantum observable using the SparsePauliOp class.
# docs: https://docs.quantum.ibm.com/api/qiskit/qiskit.quantum_info.SparsePauliOp
# <CHSH1> = <AB> - <Ab> + <aB> + <ab> -> <ZZ> - <ZX> + <XZ> + <XX>
observable1 = SparsePauliOp.from_list([("ZZ", 1), ("ZX", -1), ("XZ", 1), ("XX", 1)])

# Create a second quantum observable using the SparsePauliOp class.
# <CHSH2> = <AB> + <Ab> - <aB> + <ab> -> <ZZ> + <ZX> - <XZ> + <XX>
observable2 = SparsePauliOp.from_list([("ZZ", 1), ("ZX", 1), ("XZ", -1), ("XX", 1)])

In [ ]:
#creating a local testing code as in Testing-Locally.ipynb

# Importing Qiskit estimator to use on local computer (Not quantum computer)
from qiskit.primitives import StatevectorEstimator as LocalEstimator

#Use local estimator for expectation values without quantum computers. Seed=None sets a randomly generated seed.
estimator = LocalEstimator(seed=None)
pub = (
    chsh_circuit,  # ISA circuit
    [[observable1], [observable2]],  # ISA Observables
    individual_phases,  # Parameter values
)

#Runs the estimator to obtain simulated results.
job_result = estimator.run(pubs=[pub]).result()

#Returns the expectation values for each observable.
chsh1_est = job_result[0].data.evs[0]
chsh2_est = job_result[0].data.evs[1]

In [ ]:
#code block to receive the expectation values from a quantum computer and plot the final output

fig, ax = plt.subplots(figsize=(10, 6))

# results from hardware
ax.plot(phases / np.pi, chsh1_est, "o-", label="CHSH1", zorder=3)
ax.plot(phases / np.pi, chsh2_est, "o-", label="CHSH2", zorder=3)

# classical bound +-2
ax.axhline(y=2, color="0.9", linestyle="--")
ax.axhline(y=-2, color="0.9", linestyle="--")

# quantum bound, +-2√2
ax.axhline(y=np.sqrt(2) * 2, color="0.9", linestyle="-.")
ax.axhline(y=-np.sqrt(2) * 2, color="0.9", linestyle="-.")
ax.fill_between(phases / np.pi, 2, 2 * np.sqrt(2), color="0.6", alpha=0.7)
ax.fill_between(phases / np.pi, -2, -2 * np.sqrt(2), color="0.6", alpha=0.7)

# set x tick labels to the unit of pi
ax.xaxis.set_major_formatter(tck.FormatStrFormatter("%g $\\pi$"))
ax.xaxis.set_major_locator(tck.MultipleLocator(base=0.5))

# set labels, and legend
plt.xlabel("Theta")
plt.ylabel("CHSH witness")
plt.legend()
plt.show()